# Manual Parameter Scan

In [1]:
import itertools
import uproot
import awkward as ak
import numpy as np
import pandas as pd
from coffea import nanoevents
import vector
import fastjet
import math
import matplotlib.pyplot as plt
import matplotlib
import mplhep as hep
import hist
from hist import Hist
from rich.table import Table
from rich.console import Console

In [2]:
matplotlib.rcParams.update({'figure.max_open_warning': 0})
plt.style.use(hep.style.CMS)

In [3]:
vector.register_awkward()

In [4]:
np.set_printoptions(suppress=True)

In [5]:
def inter_isolation(leptons_1, leptons_2, dR=1.6):
    """
    Compute the inter-isolation of each particle. It is supposed to work for one particle per event. The input is:
    - leptons_1: array of leptons for isolation calculation
    - leptons_2: array of all leptons in the events
    - dR: deltaR cut for isolation calculation
    """
    a, b = ak.unzip(ak.cartesian([leptons_1, leptons_2]))
    deltar_mask = a.deltaR(b) < dR
    return (ak.sum(b[deltar_mask].pt, axis=-1) - leptons_1.pt) / leptons_1.pt

def muon_filter(events, nMuons=4):
    """
    Filter events after the TripleMu trigger.
    Cleans muons and electrons.
    Requires at least nMuons with mediumId, pt, dxy, dz, and eta cuts.
    """
    muons = events.Muon
    electrons = events.Electron
    clean_muons = (
        (events.Muon.mediumId)
        & (events.Muon.pt > 3)
        & (abs(events.Muon.dxy) <= 0.02)
        & (abs(events.Muon.dz) <= 0.1)
        & (abs(events.Muon.eta) < 2.4)
    )
    clean_electrons = (
        (events.Electron.mvaFall17V2noIso_WPL)
        & (events.Electron.pt > 3)
        & (
            abs(events.Electron.dxy)
            < 0.05 + 0.05 * (abs(events.Electron.eta) > 1.479)
        )
        & (
            abs(events.Electron.dz)
            < 0.10 + 0.10 * (abs(events.Electron.eta) > 1.479)
        )
        & ((abs(events.Electron.eta) < 1.444) | (abs(events.Electron.eta) > 1.566))
        & (abs(events.Electron.eta) < 2.5)
    )
    muons = muons[clean_muons]
    electrons = electrons[clean_electrons]
    select_by_muons = ak.num(muons, axis=-1) >= nMuons
    events = events[select_by_muons]
    muons = muons[select_by_muons]
    electrons = electrons[select_by_muons]
    return events, electrons, muons

In [6]:
lumi = 59817.406
xsec_125 = 45.2

In [7]:
# Some commone paths
xrd_path = 'root://cmseos.fnal.gov//store/user/chpapage/SUEP_low_temp/2018/NANOAOD_merged/'
pre = "step_NANOAOD_ggH-channel_mMed-125"
dataset = lambda mD, temp, decay: f"{pre}_mDark-{mD}_temp-{temp}_decay-{decay}_13TeV-pythia8.root"

In [9]:
decays_vs_mD = {
  "darkPho": [1, 2, 4, 8],
  "darkPhoHad": [1.4, 2, 4, 8],
}
ratios = [0.25, 0.5, 1, 2, 4]

In [13]:
cutflows = {}
pt_leading = []
pt_mean = []
h_i16 = []
for decay, mDs in decays_vs_mD.items():
    for mD in mDs:
        for r in ratios:
            temp = r * mD
            file = nanoevents.NanoEventsFactory.from_root(
                xrd_path + dataset(mD, temp, decay).replace('.0', ''),
                schemaclass=nanoevents.NanoAODSchema
            )
            events = file.events()
            name = f'mD-{mD}_T-{temp}_{decay}'
            name = name.replace('.0', '')
            print(f'Working on {name}')
            cutflows[name] = Hist.new.StrCategory(
                [
                    'TripleMu_5_3_3_Mass3p8_DZ',  
                    'TripleMu_10_5_5_DZ',  
                    'Logical OR',
                ], 
                name='eff', 
                label='eff',
            ).Weight()
            n_total = len(events)
            scale = lumi * xsec_125 / n_total

            cutflows[name].fill(
                ['TripleMu_5_3_3_Mass3p8_DZ'], 
                weight=(len(events[events.HLT.TripleMu_5_3_3_Mass3p8_DZ == 1]) / n_total)
            )

            cutflows[name].fill(
                ['TripleMu_10_5_5_DZ'], 
                weight=(len(events[events.HLT.TripleMu_10_5_5_DZ == 1]) / n_total)
            )

            cutflows[name].fill(
                ['Logical OR'], 
                weight=(
                    len(events[(events.HLT.TripleMu_5_3_3_Mass3p8_DZ == 1) | (events.HLT.TripleMu_10_5_5_DZ == 1)]) / n_total
                )
            )


Working on mD-1_T-0.25_darkPho
Working on mD-1_T-0.5_darkPho
Working on mD-1_T-1_darkPho
Working on mD-1_T-2_darkPho
Working on mD-1_T-4_darkPho
Working on mD-2_T-0.5_darkPho
Working on mD-2_T-1_darkPho
Working on mD-2_T-2_darkPho
Working on mD-2_T-4_darkPho
Working on mD-2_T-8_darkPho
Working on mD-4_T-1_darkPho
Working on mD-4_T-2_darkPho
Working on mD-4_T-4_darkPho
Working on mD-4_T-8_darkPho
Working on mD-4_T-16_darkPho
Working on mD-8_T-2_darkPho
Working on mD-8_T-4_darkPho
Working on mD-8_T-8_darkPho
Working on mD-8_T-16_darkPho
Working on mD-8_T-32_darkPho
Working on mD-1.4_T-0.35_darkPhoHad
Working on mD-1.4_T-0.7_darkPhoHad
Working on mD-1.4_T-1.4_darkPhoHad
Working on mD-1.4_T-2.8_darkPhoHad
Working on mD-1.4_T-5.6_darkPhoHad
Working on mD-2_T-0.5_darkPhoHad
Working on mD-2_T-1_darkPhoHad
Working on mD-2_T-2_darkPhoHad
Working on mD-2_T-4_darkPhoHad
Working on mD-2_T-8_darkPhoHad
Working on mD-4_T-1_darkPhoHad
Working on mD-4_T-2_darkPhoHad
Working on mD-4_T-4_darkPhoHad
Work

### Cutflow tables

In [14]:
table = Table(title="HLT_TripleMu trigger efficiencies")
table.add_column("Sample", justify="center", style="cyan", no_wrap=True, width=22)

year='2018'
samples = []
for decay, mDs in decays_vs_mD.items():
    for mD in mDs:
        for r in ratios:
            temp = r * mD
            samples += [f'mD-{mD}_T-{temp}_{decay}'.replace('.0', ''),]

n=3
cols_to_show = [0, 1, 2]# list(range(n,n+3))
for i in cols_to_show:
    col_name = cutflows[samples[0]].axes[0].bin(i)
    style = "green"
    if col_name == "all":
        style = "magenta"
    table.add_column(col_name, justify="center", style=style, no_wrap=True, width=22)

for sample in samples:
    string = []
    estring = []
    for i in range(cutflows[sample].size-1):
        s = f'{cutflows[sample][i].value:0.2f}'
        #es = f'{math.sqrt(cutflows[sample][i].variance):0.2f}'
        if cutflows[sample][i].value < 0.1:
            s = f'{cutflows[sample][i].value:0.3f}'
            #es = f'{math.sqrt(cutflows[sample][i].variance):0.1e}'
        string += [s]
        #estring += [es]
    table.add_row(
        sample,
        #*[f'{string[i]} ± {estring[i]}' for i in cols_to_show], 
        *[f'{string[i]}' for i in cols_to_show], 
    )

console = Console()
console.print(table)

                              HLT_TripleMu trigger efficiencies                              
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃        Sample        ┃   5_3_3_Mass3p8_DZ   ┃      10_5_5_DZ       ┃          OR          ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ mD-1_T-0.25_darkPho  │        0.000         │        0.000         │        0.000         │
│  mD-1_T-0.5_darkPho  │        0.001         │        0.002         │        0.002         │
│   mD-1_T-1_darkPho   │        0.004         │        0.006         │        0.009         │
│   mD-1_T-2_darkPho   │        0.069         │        0.030         │        0.078         │
│   mD-1_T-4_darkPho   │         0.37         │         0.14         │         0.40         │
│  mD-2_T-0.5_darkPho  │        0.002         │        0.005         │        0.006         │
│   mD-2_T-1_darkPho   │        0.022         │        0.017         │        0.031         │
│   mD-2_T-2_darkPho   │         0.19         │        0.058         │         0.20         │
│   mD-2_T-4_darkPho   │         0.42         │         0.25         │         0.46         │
│   mD-2_T-8_darkPho   │         0.43         │         0.42         │         0.52         │
│   mD-4_T-1_darkPho   │        0.063         │        0.027         │        0.073         │
│   mD-4_T-2_darkPho   │         0.25         │        0.079         │         0.27         │
│   mD-4_T-4_darkPho   │         0.45         │         0.28         │         0.50         │
│   mD-4_T-8_darkPho   │         0.43         │         0.43         │         0.51         │
│  mD-4_T-16_darkPho   │         0.27         │         0.38         │         0.40         │
│   mD-8_T-2_darkPho   │         0.41         │         0.13         │         0.42         │
│   mD-8_T-4_darkPho   │         0.51         │         0.33         │         0.55         │
│   mD-8_T-8_darkPho   │         0.44         │         0.44         │         0.52         │
│  mD-8_T-16_darkPho   │         0.29         │         0.38         │         0.41         │
│  mD-8_T-32_darkPho   │         0.18         │         0.30         │         0.31         │
│ mD-1.4_T-0.35_darkP… │        0.000         │        0.000         │        0.000         │
│ mD-1.4_T-0.7_darkPh… │        0.001         │        0.002         │        0.002         │
│ mD-1.4_T-1.4_darkPh… │        0.004         │        0.005         │        0.007         │
│ mD-1.4_T-2.8_darkPh… │        0.033         │        0.019         │        0.042         │
│ mD-1.4_T-5.6_darkPh… │        0.086         │        0.055         │         0.11         │
│ mD-2_T-0.5_darkPhoH… │        0.001         │        0.001         │        0.002         │
│ mD-2_T-1_darkPhoHad  │        0.003         │        0.006         │        0.007         │
│ mD-2_T-2_darkPhoHad  │        0.022         │        0.015         │        0.029         │
│ mD-2_T-4_darkPhoHad  │        0.069         │        0.043         │        0.087         │
│ mD-2_T-8_darkPhoHad  │        0.076         │        0.092         │         0.12         │
│ mD-4_T-1_darkPhoHad  │        0.011         │        0.010         │        0.017         │
│ mD-4_T-2_darkPhoHad  │        0.039         │        0.021         │        0.050         │
│ mD-4_T-4_darkPhoHad  │        0.072         │        0.059         │        0.096         │
│ mD-4_T-8_darkPhoHad  │        0.067         │        0.089         │         0.11         │
│ mD-4_T-16_darkPhoHad │        0.041         │        0.075         │        0.080         │
│ mD-8_T-2_darkPhoHad  │        0.073         │        0.035         │        0.086         │
│ mD-8_T-4_darkPhoHad  │        0.094         │        0.071         │         0.12         │
│ mD-8_T-8_darkPhoHad  │        0.070         │        0.092         │         0.11         │
│ mD-8_T-16_darkPhoHad │        0.045         │     